<a href="https://colab.research.google.com/github/MonitSharma/Learn-Machine-Learning-with-Me/blob/main/Counter_Factual_Explained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Counterfactual explanations
Counterfactual explanations (CEs) are an important tool from the field of explainable artificial intelligence (XAI). 
This notebook teaches what CEs are, why they are important, and provides how they can be discovered.

-----------
--------


## To begin with: What is *XAI*?
XAI is a subfield of AI concerned with developing methods to help us use AI systems in a fair, safe, and responsible manner.
To do that, XAI aims at *explaining* why an AI system (typically, actually a machine learning model) behaves the way it does.
There are two main categories of XAI methods:

1.  Methods to understand why very large and complex models, like deep neural nets and large ensembles of decision trees, come to certain decisions/predictions. 
These models are typically called *black-box* models.

2.  Methods to generate models that are so simple that they can be interpreted directly. Models of this type are, e.g., decision trees, rule sets, and equations found by symbolic regression.
These models are typically called *glass-box* models.



-----------------
-----------------




## A brief intro to CEs
CEs belong to the first category mentioned above: methods to explain black-box models.
Let us consider the case in which we have a model that is a classifier, i.e., our model is a function $$f : \Omega^d → \mathbb{C},$$
where $\Omega^d$ is our space of $d$ features (some of which are numerical and thus in $\mathbb{R}$, some of which are categorical) while $\mathbb{C}$ is the space of classes (for example for a classifier of credit risk, $\mathbb{C} = \{ \textit{High risk}, \textit{Low risk} \}$).

Say $\mathbf{x} \in \Omega^d$ is a possible input for our classifier $f$.
$\mathbf{x}$ represents a user. For example, $\mathbf{x}$ can be the:
$$\mathbf{x} = ( \textit{ age : 22, gender : Female, savings : 5.000\$, job : student, } \dots ). $$
For a given $\mathbf{x}$, $f$ will predict a certain class $c$ (e.g., "$\textit{High risk}$").
Now, a CE aims to answer the question:
"What **small change** is needed to $\mathbf{x}$ such that the new input $\mathbf{x}^\prime$ will cause $f$ to produce the desired class $c^\star$? (e.g., $f(\mathbf{x}^\prime) = \textit{Low risk}$).

A CE is a possible answer to the question above. 
For example, an answer could be that the user needs to increase their savings ($\textit{5.000\$} → {8.000\$}$) and change occupation ($ \textit{student} \rightarrow \textit{part-time employed}$).
However, a CE may also reveal that $f$ changes its prediction based on ethnicity or gender (all other features remaining the same), meaning that $f$ learned harmful biases (e.g., from historical data) that perpetuate a discrimination against minorities (unfairness).

-------------------------


Here's a simplified depiction in a 2D feature space:
![](https://drive.google.com/uc?export=view&id=1eQTEExQhIgi-2sEoCcyMELfKXACTrxAW)


-------------------
--------------------


### Seeking *small* changes to $x$

We seek *small* changes to $x$ to observe how $f$ behaves in the neighborhood of an input to gain information on what the decision boundary looks like in that area. 
Moreover, a very interesting property of CEs is that they prescribe a possible intervention that the user may actually want to pursue!
Thus, we wish that the cost of intervention is small for the user. 
This means that $\mathbf{x}^\prime$ needs to be as close as possible to $\mathbf{x}$, under some meaningful distance function $\delta$ that captures the cost of intervention.


---------------------------
--------------------



## Additional reading material
An excellent and beginner-friendly starting point is the book by Christoph Molnar: "Inteprable ML Book".
Here's a direct link to his chapter on CEs (co-written by Susanne Dandl): https://christophm.github.io/interpretable-ml-book/counterfactual.html




---------------------------------
--------------------------------


## Note: CEs vs adversarial examples
CEs are similar to adversarial examples (AEs). In both cases, one searches for changes to the input $x$ that trigger a change to the prediction made by $f$. However, CEs are intended to explain $f$ and not to fool it!

## Let's get started


In this notebook we simulate a financial credit risk situation, in which a black-box model (we will be using a random forest) has been trained to tell which users are at high or low risk of default (i.e., become unable of paying back the credit given by the bank).
We will then use a CE discovery algorithm to see how an user can change their (unfavorable) situation (i.e., $f(x)=\text{high risk}$).

## Set up the Libraries required and set the random seed value

In [1]:
import numpy as np   # numerical calculations
import pandas as pd  # data handling


# use sklearn for ML
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score


SEED = 135
np.random.seed(SEED)   # we set this to reproduce this result again

### Load data
We load the data set **"South German Credit"**, which concerns learning a model of whether providing a financial credit to a user may be safe or risky.
See [this](https://archive.ics.uci.edu/ml/datasets/South+German+Credit+%28UPDATE%29) for more info.

We get this data from the repo of CoGS, a baseline algorithm for the discovery of CEs (more details later). 

In [ ]:
# clone the mentioned repo, and install the repo
!git clone https://github.com/marcovirgolin/cogs
%cd /content/cogs
!pip install .

Load the data and preprocess it a bit.

In [6]:
# Load data set & do some pre-processing
df = pd.read_csv("south_german_credit.csv")
df.drop("account_check_status",axis=1,inplace=True)
categorical_feature_names = ['purpose', 'personal_status_sex',
    'other_debtors', 'other_installment_plans', 'telephone', 'foreign_worker']
# Note: some other features are indices (categories in which the order matters), treated as numerical here for simplicity
label_name = 'credit_risk'
desired_class = 1 # this means "low risk"

for feat in categorical_feature_names: # convert categorical features into integer codes
    df[feat] = pd.Categorical(df[feat])
    df[feat] = df[feat].cat.codes 
feature_names = list(df.columns)
feature_names.remove(label_name)

print("Num. features: {}, feature names: {}".format(len(feature_names), feature_names))

# Prepare data to be in numpy format, as typically used to train a scikit-learn model
X = df[feature_names].to_numpy()
y = df[label_name].to_numpy().astype(int)
# Assume we have a specific train & test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=SEED)

Num. features: 19, feature names: ['duration_in_month', 'credit_history', 'purpose', 'credit_amount', 'savings', 'present_emp_since', 'installment_as_income_perc', 'personal_status_sex', 'other_debtors', 'present_res_since', 'property', 'age', 'other_installment_plans', 'housing', 'credits_this_bank', 'job', 'people_under_maintenance', 'telephone', 'foreign_worker']


### Train the model
Here we train the model, but in a practical situation we may assume that the model has already been trained (and is, e.g., property of the bank that assesses to whether to award the credit or not).

We use `random_forest` because it is quick and easy. However, you can use any model you like, such as a deep neural net. 
As classicly done in XAI litereature, we call this model a *black-box model*. 

In [7]:
# Train black-box model (bbm)
bbm = RandomForestClassifier(random_state=SEED, class_weight="balanced", min_samples_leaf=25)
bbm.fit(X_train, y_train)
# note: we do not one-hot encode multi-category features here for simplicity

RandomForestClassifier(class_weight='balanced', min_samples_leaf=25,
                       random_state=135)

Let's check that the model has a decent accuracy 
(Note: not really needed for the purpose of CEs)

In [8]:
print("acc:{:.3f}, bal.-acc:{:.3f}".format(accuracy_score(y_test, bbm.predict(X_test)), balanced_accuracy_score(y_test, bbm.predict(X_test))))

acc:0.670, bal.-acc:0.676


### Pick the user
Next, we simulate to have a user for whom the decision of the black-box model is the undesired one. 
For example, let's pick the last point in the test set for which the prediction is unfavourable.

In [9]:
# Let's consider, e.g., the last test sample for which an undesired decision is given
p = bbm.predict(X_test)
idx = np.argwhere(p != desired_class).squeeze()[-1]
x = X_test[idx] # this is our unhappy user!

# show features of this user
print("Description of x:")
for i, feat_name in enumerate(feature_names):
  print(" ", feat_name+" "*(30-len(feat_name)), x[i])

Description of x:
  duration_in_month              6
  credit_history                 0
  purpose                        0
  credit_amount                  1204
  savings                        2
  present_emp_since              3
  installment_as_income_perc     4
  personal_status_sex            2
  other_debtors                  0
  present_res_since              1
  property                       4
  age                            35
  other_installment_plans        0
  housing                        1
  credits_this_bank              1
  job                            3
  people_under_maintenance       2
  telephone                      0
  foreign_worker                 0


### CE discovery algorithm
We use the library CoGS to find a CE.
CoGS (Counterfactual Genetic Search) is a relatively quick to run and easy to use library that makes no assumptions on the black-box model $f$ (e.g., it does not require linearity nor gradients to work). 
Moreover, CoGS can handle both numerical and categorical features.


### Setting up the search space
To set up the space in which CoGS searches, we must provide:
1) Intervals within which the search takes place (for categorical features, which categories are possible)
2) The indices of categorical features (for CoGS to know which are categorical and which are numerical)
3) Optional plausibility constraints to ensure that the discovered CE can be realized (e.g., the age of a person cannot decrease)

All of these three must be provided as lists that have the same order, in particular the order used to list the feature in `X_train` and `X_test`.

In [10]:
# Set up search bounds
feature_intervals = list()
for i, feat in enumerate(feature_names):
  if feat in categorical_feature_names:
    interval_i = np.unique(X_train[:,i])
  else:
    interval_i = (np.min(X_train[:,i]), np.max(X_train[:,i]))
  feature_intervals.append(interval_i)

# Set up which feature indices are categorical
indices_categorical_features = [i for i, feat in enumerate(feature_names) if feat in categorical_feature_names]

# Let's also set up a plausibility constraint for the feature "age" (can only increase)
# and one for foreign worker (cannot change, must stay equal to what it is)
pcs = ['>=' if feat=='age' else ('=' if feat=='foreign_worker' else None) for feat in feature_names]

## Hyper parameters
We can now setup the hyper-parameters of CoGS, and then run the search!
We put some comments to explain what they mean in the code below.

As distance $\delta$, here we use Gower's distance, which takes into account both numerical differences and categorical mismatches ,see [this](https://christophm.github.io/interpretable-ml-book/counterfactual.html#method-by-dandl-et-al.).

-------


In a genetic algorithm, the quality of solutions is measured in terms of *fitness*, where normally higher is better.
Thus the fitness used here is set to be the opposite of Gower's distance.

In [11]:
from cogs.evolution import Evolution
from cogs.fitness import gower_fitness_function

cogs = Evolution(
        ### hyper-parameters of the problem (required!) ###
        x=x,  # the starting point aka unhappy user
        fitness_function=gower_fitness_function,  # a classic fitness function for counterfactual explanations
        fitness_function_kwargs={'blackbox':bbm,'desired_class': desired_class},  # these must be passed for the fitness function to work
        feature_intervals=feature_intervals,  # intervals within which the search operates
        indices_categorical_features=indices_categorical_features,  # the indices of the features that are categorical
        plausibility_constraints=pcs, # can be "None" if no constraints need to be set
        ### hyper-parameters of the evolution (all optional) ###
        evolution_type='classic',       # the type of evolution, classic works quite  well
        population_size=1000,           # how many candidate counterfactual examples to evolve simultaneously
        n_generations=25,               # number of iterations for the evolution
        selection_name='tournament_4',  # selection pressure
        init_temperature=0.8, # how "far" from x we initialize
        num_features_mutation_strength=0.25, # strength of random mutations for numerical features
        num_features_mutation_strength_decay=0.5, # decay for the hyper-param. above
        num_features_mutation_strength_decay_generations=[10,15,20], # when to apply the decay
        ### other settings ###
        verbose=True  # logs progress at every generation 
)

🏃

In [12]:
cogs.run()

generation: 1 best fitness: -0.2396228206008183 avg. fitness: -0.6730473827620418
generation: 2 best fitness: -0.2396228206008183 avg. fitness: -0.45820597393016405
generation: 3 best fitness: -0.2396228206008183 avg. fitness: -0.4011106472660075
generation: 4 best fitness: -0.22754762910530613 avg. fitness: -0.34520111096264683
generation: 5 best fitness: -0.1470669616557555 avg. fitness: -0.29425159612096013
generation: 6 best fitness: -0.14065937232592834 avg. fitness: -0.25342080548999935
generation: 7 best fitness: -0.10759722193658999 avg. fitness: -0.21507342631622414
generation: 8 best fitness: -0.09987989964764468 avg. fitness: -0.18177387222206162
generation: 9 best fitness: -0.07570383688339692 avg. fitness: -0.14952872918991617
generation: 10 best fitness: -0.06746061714537081 avg. fitness: -0.12662933869575957
generation: 11 best fitness: -0.04942280129404737 avg. fitness: -0.10802405368291662
generation: 12 best fitness: -0.04942280129404737 avg. fitness: -0.0976667205197

## Counterfactual explanation
Now that CoGS has terminated, we can look at its result.
The field `cogs.elite` contains the best-found counterfactual example, i.e., a point `x'` for which `bbm(x')=desired_class`.
The respective counterfactual explanation is simply `x'-x` (there exist more involved definitions of counterfactual explanations, here we use this simple one).
Let's take a look at what the user needs to do to obtain the desired class, i.e., be granted the loan.

In [13]:
from pandas.core.arrays import categorical
# Get the best-found counterfactual example (called elite)
cf_example = cogs.elite
cf_explanation = cogs.elite - x

# Show counterfactual explanation
if bbm.predict([cf_example])[0] == desired_class:
  print("Success! Here's the explanation:")
  for i, feat in enumerate(feature_names):
    if cf_explanation[i] != 0:
      print(" Feature '{}' should change from '{}' to '{}'".format(feat, np.round(x[i],3), np.round(cf_example[i],3)))
else:
  print("Failed to find a counterfactual explanation for the desired class :(")

Success! Here's the explanation:
 Feature 'credit_history' should change from '0' to '3.502'
